In [42]:
import pandas as pd
import os
import h5py

In [ ]:
#!!!Должны делать матрицу по чипам, чтобы совпадали колонки в матрицах(по биосемплам)!!!

In [37]:
targets = [("H3K27ac", ""), ("H3K27me3", ""), ("H3K4me1", "_narrow"), ("H3K36me3", ""), ("H3K4me2", "_narrow"), ("H3K4me3", "_narrow"), ("H3K79me2", ""), 
           ("H3K9ac", "_narrow"), ("H3K9me3", ""), ("H4K20me1", "")]

In [1]:
def getFiles(target):
    biosamples = []
    for dirpath, dirnames, filenames in os.walk("/data/mazurovev/" + target[0] + "/"):
        for filename in sorted([f for f in filenames if not f.endswith("peaks.bed")])[:1]:
            biosamples.append(dirpath + "/" + filename)
    biosamples = [f.split("/")[-2] for f in sorted(biosamples)]
    
    files = []
    for dirpath, dirnames, filenames in os.walk("/data/mazurovev/RNA-Seq/"):
        if(dirpath.split("/")[-1] in biosamples):
            for filename in sorted([f for f in filenames])[:1]:
                files.append(dirpath + "/" + filename)
                
    return sorted(files)

In [27]:
def makeRNAMatrix(files, target):
    rna_dict_for_df = {}
    for f in files:
        tsv = pd.read_csv(f, sep="\t")
        counts = tsv["expected_count"].astype(int)
        counts.index = tsv['gene_id']
        rna_dict_for_df["/".join(f.split("/")[-2:])] = counts
        
    matrix = pd.DataFrame(rna_dict_for_df)
    ensg_matrix = matrix.filter(regex='ENSG*', axis=0)
    del ensg_matrix.index.name
    ensg_matrix = ensg_matrix.reindex(sorted(ensg_matrix.columns), axis=1)
    
    ensg_matrix.to_csv("../all_marks/" + target[0] + "/rna_matrix.csv", sep="\t")

In [6]:
for target in targets:
    print("make RNA matrix for " + target[0] + "...")
    makeRNAMatrix(getFiles(target), target)

make RNA matrix for H3K4me1 ...
make RNA matrix for H3K4me2 ...
make RNA matrix for H3K4me3 ...
make RNA matrix for H3K79me2 ...
make RNA matrix for H3K9ac ...
make RNA matrix for H3K9me3 ...
make RNA matrix for H4K20me1 ...


In [30]:
def rnaMatrixForMethylation():
    with h5py.File("../all_marks/" + "methylation" + "/signal_matrix_outer_filtered.hdf5", 'r') as f:
        m_files = f['files_names'][:]
        
    biosamples = [f.split("/")[-2] for f in m_files]
    files = []

    for path in ["/data/mazurovev/RNA-Seq/" + b + "/" for b in biosamples]:
        for filename in sorted([f for f in os.listdir(path)])[:1]:
            files.append(path + filename)
            
    makeRNAMatrix(files, ("methylation", ""))

In [31]:
rnaMatrixForMethylation()

In [40]:
d = {}
for t in targets:
    d[t[0]] = pd.read_csv("../all_marks/" + t[0] + "/rna_matrix.csv", sep="\t", index_col=0).shape

In [41]:
d

{'H3K27ac': (60725, 49),
 'H3K27me3': (60725, 50),
 'H3K4me1': (60725, 51),
 'H3K36me3': (60725, 53),
 'H3K4me2': (60725, 19),
 'H3K4me3': (60725, 59),
 'H3K79me2': (60725, 20),
 'H3K9ac': (60725, 19),
 'H3K9me3': (60725, 50),
 'H4K20me1': (60725, 19)}

In [43]:
test = pd.read_csv("../all_marks/" + "H3K27ac" + "/rna_matrix.csv", sep="\t", index_col=0)

In [32]:
matrix = pd.read_csv("../all_marks/" + "methylation" + "/rna_matrix.csv", sep="\t", index_col=0)

In [35]:
matrix.head()

,CL:0000182/ENCFF072XSA.tsv,CL:0000192/ENCFF401KXP.tsv,CL:0000515/ENCFF803CZZ.tsv,CL:0002327/ENCFF805ZGF.tsv,EFO:0001187/ENCFF041HHL.tsv,EFO:0001196/ENCFF353SBP.tsv,EFO:0002067/ENCFF104VTJ.tsv,EFO:0002784/ENCFF306TLL.tsv,EFO:0003042/ENCFF093NEQ.tsv,EFO:0005723/ENCFF092TYT.tsv,...,UBERON:0001323/ENCFF046AFQ.tsv,UBERON:0002046/ENCFF083NSQ.tsv,UBERON:0002048/ENCFF573QQU.tsv,UBERON:0002084/ENCFF862LZL.tsv,UBERON:0002106/ENCFF474KYX.tsv,UBERON:0002240/ENCFF126UNL.tsv,UBERON:0002367/ENCFF499KKR.tsv,UBERON:0002369/ENCFF212SUD.tsv,UBERON:0004264/ENCFF187DQQ.tsv,UBERON:0008952/ENCFF042JPR.tsv
ENSG00000000003.14,7390,5119,1792,1589,1955,1677,0,1,397,1493,...,400,435,1572,71,206,1953,935,467,595,280
ENSG00000000005.5,50,0,0,0,0,0,0,0,22,0,...,1,0,2,2,0,43,0,0,17,0
ENSG00000000419.12,1718,3029,1882,1558,538,1591,1911,2059,484,1196,...,226,287,437,217,223,562,861,397,639,326
ENSG00000000457.13,639,888,690,929,126,1042,498,1053,137,606,...,312,472,1001,143,283,941,1394,425,968,341
ENSG00000000460.16,535,1075,277,295,580,1109,1332,1288,420,460,...,173,127,397,7,147,584,291,125,123,57


In [33]:
matrix.shape

(60725, 27)

In [11]:
matrix.values[100:105, 13:18].T

array([[ 714.99,  863.3 ,   19.65, 1267.41,  614.06],
       [2079.6 ,  148.88,    0.  , 2948.34, 1198.98],
       [ 518.37,  139.44,  351.05,  966.21,  792.32],
       [2138.4 ,  751.08,    0.  , 1579.34,  485.53],
       [1270.25,  822.4 ,   41.55, 3680.31,  422.45]])

In [12]:
pd.DataFrame(data=matrix.values[100:105, 13:18].T, index=['sample_1', "sample_2", "sample_3", "sample_4", "sample_5"], columns=['gene_1', "gene_2", "gene_3", "gene_4", "gene_5"])

,gene_1,gene_2,gene_3,gene_4,gene_5
sample_1,714.99,863.30,19.65,1267.41,614.06
sample_2,2079.60,148.88,0.00,2948.34,1198.98
sample_3,518.37,139.44,351.05,966.21,792.32
sample_4,2138.40,751.08,0.00,1579.34,485.53
sample_5,1270.25,822.40,41.55,3680.31,422.45
